In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('hate_speech_preprocessed.csv')

In [4]:
train_df.rename(columns = {'comments_regex_spellcheck_pos' : 'comments_pos'}, inplace=True)

In [7]:
train_df = train_df[['news_title', 'comments', 'comments_pos', 'bias', 'contain_gender_bias', 'hate']]

In [8]:
train_df.head()

,news_title,comments,comments_pos,bias,contain_gender_bias,hate
0,"""밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]""",(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,현재 호텔 주인 심정 아 18 나 마르 하늘 벼락 맞 고 호텔 망하 게 생기 는데 ...,others,False,hate
1,"""'연중' 故 전미선, 생전 마지막 미공개 인터뷰…환하게 웃는 모습 '먹먹'[종합]""",....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,한국 적 미인 대표 적 분 너무나 곱 고 아름답 모습 그 모습 뒤 슬픔 미 처 알 ...,none,False,none
2,"""[단독] 잔나비, 라디오 출연 취소→'한밤' 방송 연기..비판 여론 ing(종합)""","...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",못 되 넘 들 남 고통 즐기 넘 들 이제 마땅 하 처벌 받 아야지 그 렇 어야 공정...,none,False,hate
3,"""'아스달 연대기' 장동건-김옥빈, 들끓는 '욕망커플'→눈물범벅 '칼끝 대립'""","1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",1 2 화 어설 푸 는데 3 4 화 지나 아서 갈 수 록 너무 재밌 데,none,False,none
4,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1 사람 얼굴 손톱 긁 것 인격 살해 고 2 동영 상 몰카 냐 메 걸리안 들 생각 없 노,gender,True,hate


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(train_df.comments_pos) # khaiii로 형태소 분석한 코멘트를 tf-idf 수행
print(tfidf_matrix.shape)

(7893, 11978)


In [10]:
from sklearn.metrics.pairwise import linear_kernel


cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) # 코사인 유사도

In [11]:
# 코멘트에 관한 인덱스 테이블 구축
indices = pd.Series(train_df.index, index=train_df.comments_pos).drop_duplicates()

In [12]:
def comments_cos(comments, df, cosine_sim=cosine_sim): # inputs: str : comments, pandas df: dataframe
    #입력한 코멘트로 부터 인덱스 가져오기
    idx = indices[comments]

    # 모든 코멘트에 대해서 해당 코멘트와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 코멘트들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 3개의 코멘트를 받아옴
    sim_scores = sim_scores[1:4]

    # 가장 유사한 3개 코멘트의 인덱스 받아옴
    comment_indices = [i[0] for i in sim_scores]
    
    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = df.iloc[comment_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]
    
    # 필요없는 컬럼 삭제
    del result_df['comments_pos']

    # 가장 유사한 3개의 코멘트을 리턴
    return result_df

In [13]:
train_df[['comments', 'hate']].iloc[330]

comments    갈수록 종답이네
hate            none
Name: 330, dtype: object

In [14]:
comments_cos(train_df['comments_pos'][330], train_df) # 샘플 아웃풋

,news_title,comments,bias,contain_gender_bias,hate,score
325,"""[DA:리뷰] “넌 내 모든 것”…‘김비서’ 박서준♥박민영, 고통 뛰어넘은 사랑(...",갈수록 노잼,none,False,none,0.637108
326,"""[종합]""""아빠라서 죄송"""" '하나뿐인내편' 윤진이, 최수종X유이 부녀사이 폭로""",갈수록 더하네,none,False,offensive,0.637108
329,"""'나혼산' 헨리, 첫사랑 바이올린 천만원에 '안녕'…이시언, 첫 해외팬미팅 '울컥...",갈수록 재미 없어져,none,False,none,0.505136


### Case 1. All labels are the same

In [21]:
train_df[['comments','bias','contain_gender_bias','hate']].iloc[7887] # 원래 코멘트, none으로 분류 되어있음

comments               힘내세요.저두 이틀전 사촌오빠를 보내고 왔는데 맘이 추스려지지 않내요
bias                                                     none
contain_gender_bias                                     False
hate                                                     none
Name: 7887, dtype: object

In [22]:
comments_cos(train_df['comments_pos'][7887], train_df) 

,news_title,comments,bias,contain_gender_bias,hate,score
2760,"""[종합] 김민경 남동생, 9일 사망 비보→애도·위로 물결…'맛녀석' """"정상 방송""""""",민상아 처낟 잘보내고 와,none,False,none,0.345514
2397,"""'며느리' 민지영, 유산 고백 """"내가 아이 못 지킨 것 같다"""" 눈물 펑펑""",마흔이 요즘은 늦은 나이 아니예요힘내시고 몸 잘 추스리시길,none,False,none,0.269861
6675,"""정다은, 원호 채무 불이행 폭로...""""법적조치 검토""""vs변호사 문자 공개 [종합]""",제일 예쁜 마음 원호 오빠... 제일 귀여운 토끼 원호 오빠 제일 착한 사람 원호 ...,none,False,none,0.266653


In [23]:
label_01 = comments_cos(train_df['comments_pos'][7887], train_df) 

In [24]:
label_list_01 = label_01['hate'].tolist() # 라벨 리스트 화
label_list_01

['none', 'none', 'none']

In [25]:
len(set(label_list_01)) == 1 # 모든 라벨이 같을 경우

True

In [26]:
label_list_01[0] # 그 라벨을 리턴, 아무거나 리턴 해도 상관없기에 편의상 0번째 인덱스로 설정

'none'

### Case 2. Duplicate Labels

In [126]:
train_df[['news_title', 'comments','bias','contain_gender_bias','hate']].iloc[2245]

news_title              "[단독] 김민서 오늘(17일) 결혼, 미국서 신혼 생활 시작"
comments               듣보잡 미국가서 결혼하고 산다는거 하루쟁일 생중계네 톱스타도아니고
bias                                                 others
contain_gender_bias                                   False
hate                                              offensive
Name: 2245, dtype: object

In [125]:
comments_cos(train_df['comments_pos'][2245], train_df) 

,news_title,comments,bias,contain_gender_bias,hate,score
2246,"""[POP이슈]'일본→중국→바티칸' 강한나♥왕대륙, 세 번째 열애설(종합)""",듣보잡. 노잼,none,False,offensive,0.398685
745,"""‘부모님 사기설’ 마이크로닷 측 “확인결과 사실무근, 명예훼손 법적대응 준비”""",그건 모르겠고 갑자기 이 듣보잡이 쳐나대는게꼴보기싫음,others,False,hate,0.238098
515,"""[공식]'5년째 열애' 이정재♥임세령, 오늘(1일) 나란히 해외 출국…소속사 """"...",결혼도안하고 뭐하는짓?,others,False,offensive,0.211598


In [127]:
label_02 = comments_cos(train_df['comments_pos'][2245], train_df)

In [128]:
label_list_02 = label_02['hate'].tolist() # 라벨 리스트 화
label_list_02

['offensive', 'hate', 'offensive']

In [129]:
len(set(label_list_02)) == 2 # # 중복 라벨

True

In [130]:
[x for x in label_list_02 if label_list_02.count(x) > 1][0] # 중복 라벨 출력

'offensive'

### Case 3. All labels are distinct

In [179]:
train_df[['news_title', 'comments','bias','contain_gender_bias','hate']].iloc[60]

news_title             "[Oh!쎈 이슈] 추자현, 우효광과 외식→'의식불명설' 종결..소속사 ""조리원서...
comments               2천만원짜리 조리원하며고기집 외식하며...참 가지가지 한다ㅋㅋㅋㅋㅋ관심병으로 더아프...
bias                                                              others
contain_gender_bias                                                False
hate                                                           offensive
Name: 60, dtype: object

In [178]:
comments_cos(train_df['comments_pos'][60], train_df) 

,news_title,comments,bias,contain_gender_bias,hate,score
224,"""허지웅, 악성림프종 혈액암 투병 고백→방송 활동 올스톱 """"치료 전념""""[종합]""",ㅋㅋㅋ 개꼬시다,none,False,offensive,0.212234
7416,"""정웅인 딸 삼윤이 5년만 폭풍성장 """"놀이기구 몇개 타지도 못했었는데..""""""",폭풍 짜리몽땅 됬노‼️‼️‼️,others,False,hate,0.202414
4559,"""'윤종신♥'전미라 """"딸 라임이의 매력, 꾸미지 않은 자유분방함""""""",엄마닮아서 천만다행~!!,none,False,none,0.195253


In [181]:
label_03 = comments_cos(train_df['comments_pos'][60], train_df) 

In [182]:
label_list_03 = label_03['hate'].tolist()
label_list_03

['offensive', 'hate', 'none']

In [183]:
len(set(label_list_03)) == 3 # 중복 되는 값이 없으면, 일단은 부정적인 코멘트라고 가정하고 offensive나 hate으로 분류
                           # none에 threshold값을 주면 더 좋을 것 같다는 생각 
                           # 예 : none의 score가 0.5이상이라면 중복 되는 label이 없어도 none 으로 분류 

True

In [188]:
label_list_03['hate']

['offensive', 'hate', 'none']

In [189]:
label_index = label_03[label_03['hate'] != 'none'].index[0] # 부정적 코멘트중에 score 가 제일 높은 것의 인덱스 고름

In [191]:
label_03['hate'][label_index] # 예측 결과 

'offensive'

### 분류 실패

In [57]:
train_df[['news_title', 'comments','bias','contain_gender_bias','hate']].iloc[7293]

news_title             '김생민 영수증' 슬리피 전재산 7만원vs허세 명품중독[종합]
comments                          타워펠리스 관리비 월 100마넌 ᆢ 스튜핏
bias                                                 none
contain_gender_bias                                 False
hate                                            offensive
Name: 7293, dtype: object

In [58]:
comments_cos(train_df['comments_pos'][7293], train_df) # hate으로 분류 되서 실패

,news_title,comments,bias,contain_gender_bias,hate,score
4399,"""[단독]구하라 측 """"수면장애·소화불량 치료 차 병원 방문""""""",얜 10000% 스폰있다.,others,False,hate,0.245919
377,"""[SC무비] 이승기X심은경 `궁합` 100만 돌파 """"로맨스 잔혹사 끊었다""""""",강철비가 100%라면 신과함께는 85%정도 재밌었다. 이거는 어떠냐,none,False,none,0.145099
5279,"""""""29kg 감량"""" 홍지민, 눈으로 확인한 3단 변화 [종합]""",요요 100퍼 역할안들어온다고 핑계대면서 폭식 ㅋㅋ,none,False,offensive,0.134970


In [70]:
train_df[['news_title', 'comments','bias','contain_gender_bias','hate']].iloc[8]

news_title             """故설리 부재 하에 지속 어려워""…'악플의밤' 측이 폐지를 결정한 이유 [종합]"
comments                             10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐
bias                                                               none
contain_gender_bias                                               False
hate                                                          offensive
Name: 8, dtype: object

In [69]:
comments_cos(train_df['comments_pos'][8], train_df) # None으로 분류 되서 실패

,news_title,comments,bias,contain_gender_bias,hate,score
1826,"""'사풀인풀' 반성 없는 조우리…오민석♥조윤희, 한집살이 시작 [종합]""",느무 재미 없다.,none,False,none,0.368503
6334,'강식당2'는 경주에서 떡볶이?..핫한 관심→벌써 시작된 스포와의 전쟁 [종합],재미 1도 없다,none,False,none,0.368503
6760,"""""""180cm 괴어 영접""""'전설의 빅피쉬' 아마존 그랜드슬램 '대성공' (ft....",졸 재미없겠다~~,none,False,none,0.368503


In [76]:
train_df[['news_title', 'comments','bias','contain_gender_bias','hate']].iloc[125]

news_title             "트와이스 미나, 韓 입국에 활동 복귀설+눈물..JYP 측 ""일정 참여 NO""[...
comments                                             No no japan!!! 필요없다
bias                                                              others
contain_gender_bias                                                False
hate                                                                hate
Name: 125, dtype: object

In [77]:
comments_cos(train_df['comments_pos'][125], train_df)  # 가장 유사도 높은것은 거의 비슷한 덧글인데 offensive라고 되어있음

,news_title,comments,bias,contain_gender_bias,hate,score
6118,"""트와이스 미나, 韓 입국에 활동 복귀설+눈물..JYP 측 """"일정 참여 NO""""[...",일본 NO,others,False,offensive,0.670513
7452,"""조민기, 사망 전 음성공개 """"딸 대학원 입학, 신경쓰이지 않게…""""""",필요의 악 여자,gender,True,hate,0.272290
2347,"""[SC초점] """"여론이 나쁜사람 만들어""""…'성추문' 김건모, 잘못된 만남의 핑계...",룸 간게 잘못은 아닐지 몰라도 그러면 결혼을 하지 말아야지... 노총각이 아니라 N...,gender,True,offensive,0.239675


In [192]:
def predict_label(comments, df, cosine_sim=cosine_sim): # inputs: comments : str, df :pd.dataframe
    #입력한 코멘트로 부터 인덱스 가져오기
    idx = indices[comments]

    # 모든 코멘트에 대해서 해당 코멘트와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 코멘트들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 코멘트를 받아옴
    sim_scores = sim_scores[1:4]

    # 가장 유사한 10개 코멘트의 인덱스 받아옴
    comment_indices = [i[0] for i in sim_scores]
    
    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = df.iloc[comment_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]

    # hate 컬럼의 라벨 값들을 리스트로 변환후 label_list 변수에 저장
    label_list = result_df['hate'].tolist()
    if len(set(label_list)) == 1:
        pred = label_list[0]
    
    elif len(set(label_list)) == 2:
        pred = [x for x in label_list if label_list.count(x) > 1][0]
    
    else:
        label_index = result_df[result_df['hate'] != 'none'].index[0]
        pred = result_df['hate'][label_index]
          
    return pred

In [200]:
train_df[['comments', 'hate']].iloc[1245]

comments    김지원빼고 다른 배우들 다 발연기같음....
hate                            none
Name: 1245, dtype: object

In [201]:
comments_cos(train_df['comments_pos'][1245], train_df)

,news_title,comments,bias,contain_gender_bias,hate,score
7091,"""[종합]""""23년 만에 주연""""…'천화' 이일화, 덕선 엄마의 과감한 변신""",참 예쁜 배우. 연기 잘하는 배우.,none,False,none,0.490285
1932,'조선명탐정' 주말 박스오피스 1위 기록…적수없는 흥행세,다필요없고 김지원이 진짜 이쁘더라,none,False,none,0.470539
6865,"""[단독]'쥬라기 월드2' 감독, 韓 역대 최고 오프닝에 깜짝 내한""",지가 배우가 먼 내한 ㅋ,none,False,offensive,0.417219


In [202]:
predict_label(train_df['comments_pos'][1245], train_df)

'none'

In [206]:
train_df[['comments', 'hate']].iloc[1770]

comments        누구냐 넌
hate        offensive
Name: 1770, dtype: object

In [ ]:
preds = []

for i in range(len(dev_df)):
    pred = predict_label(train_df['comments_pos'][i], train_df)
    preds.append(pred)
    print('success')
    print(len(preds))